In [28]:
from rank_bm25 import BM25Okapi
from trec_cds.data.parsers import parse_clinical_trials_from_folder, parse_topics_from_xml
import spacy
import pickle
import tqdm
import numpy as np
import json

In [5]:

    folder_name = "../data/external/ClinicalTrials"
    first_n = 2000

    cts = parse_clinical_trials_from_folder(folder_name=folder_name, first_n=first_n)
    # sample

In [3]:
    nlp = spacy.load("en_core_web_sm")

In [6]:
cts

 ...]

In [7]:
    cts_tokenized = []
    for clinical_trial in tqdm.tqdm(cts):
        if clinical_trial.criteria is None or clinical_trial.criteria.strip() == "":
            doc = nlp('empty')
            print("empty")
            print(clinical_trial.summary)
        else:
            doc = nlp(clinical_trial.criteria)
        cts_tokenized.append([word.text for word in doc])

 24%|██▍       | 490/2000 [00:15<00:32, 46.13it/s]

empty
None


 51%|█████▏    | 1025/2000 [00:32<00:26, 36.48it/s]

empty
None


 55%|█████▌    | 1109/2000 [00:35<00:20, 43.58it/s]

...

 95%|█████████▍| 1896/2000 [00:59<00:02, 39.57it/s]

empty
None


 98%|█████████▊| 1957/2000 [01:01<00:01, 34.56it/s]

empty
None


100%|██████████| 2000/2000 [01:03<00:00, 31.65it/s]


In [8]:
    bm25 = BM25Okapi(cts_tokenized)
    print(bm25)

In [11]:
    topic_file = "../data/external/topics2021.xml"
    topics = parse_topics_from_xml(topic_file)

In [27]:
    output_scores = {}
    for topic in tqdm.tqdm(topics):
        topic_scores = {}
        doc = nlp(topic.text)
        doc_scores = bm25.get_scores([word.text for word in doc])
        # print(doc_scores)

        for index, score in zip(np.argsort(doc_scores)[-1000:],np.sort(doc_scores)[-1000:]):
            topic_scores[cts[index].org_study_id] = score

        output_scores[topic.number] = topic_scores

100%|██████████| 75/75 [00:05<00:00, 12.73it/s]


In [26]:
output_scores

{'1': {'WIDE&NARROW': 96.22732679125106,
  'InCaveoDanielandDaniel2021': 96.25692064330335,
  'MT & TE & 1ry KOA': 96.26365574254082,
  '2021P000704': 96.30626740205453,
  'Allo-20-001': 96.35555911559632,
  '2018/1763': 96.43856384943871,
  '29BRC17.0219': 96.49947418816618,
  'CAP-China PCT-AECOPD': 96.50036299099766,
  'MREC: 2021116-9722': 96.53844624137064,
  'CREX-006': 96.58627484445661,
  'NL76607.091.21': 96.60439176083855,
  'STU00213159': 96.60975972114429,
  '389:1/2020': 96.65786344196668,
  'CON215044': 96.6855510873285,
  '2007882387': 96.73939173796754,
  '2021/1839': 96.74711010629278,
  'MenoufiaU112019': 96.78544612978365,
  'C92F2628X5': 96.85162662954689,
  'STUDY00012537': 96.94818824094457,
  'IM039-004': 96.98202806788187,
  '208': 97.03665461064773,
  'APHP200135': 97.07218284521925,
  'TCAHCM': 97.14662461315551,
  'RECHMPL21_0021': 97.16552875526575,
  'PSMA RGS': 97.25812916832857,
  '17277': 97.35317638057114,
  'HBI-3000-402': 97.38829184736188,
  '2007030

In [29]:
with open("../data/processed/bm25-scores.json", 'w') as fp:
    json.dump(output_scores, fp)